In [2]:
import re

from splink.duckdb.duckdb_linker import DuckDBLinker
import splink.duckdb.duckdb_comparison_library as cl

from path_fns.filepaths import (
    FINAL_CORRUPTED_OUTPUT_FILES_BASE,
    DEDUPE_OUTPUTS_FILES_BASE,
    TRANSFORMED_MASTER_DATA_ONE_ROW_PER_PERSON
)

from synthetic_data.prepare import (
    prepare_df
)

from synthetic_data.random_match import (
    get_prob_two_rnd_recs_are_match
)

import pandas as pd
import numpy as np
import os
from pathlib import Path

from graphs.linkagegraph import LinkageGraph

In [9]:
dedupe_data_filename = "max_corruptions-50_prob_mult-1.0_set-1.parquet"

dedupe_data_path = os.path.join(
        DEDUPE_OUTPUTS_FILES_BASE, 
        dedupe_data_filename
    )

dedupe_records = pd.read_parquet(dedupe_data_path)
len(dedupe_records)


53238008

In [14]:
dedupe_records.head(5)

,match_weight,match_probability,unique_id_l,unique_id_r,bf_given_name,bf_family_name,bf_dob_d,bf_dob_m,bf_dob_y,bf_gender,cluster_l,cluster_r
0,16.291708,0.999988,202300000,202300015,109.34661,823.164741,16.559164,9.255721,61.778836,1.298994,20230,20230
1,16.291708,0.999988,202300001,202300015,109.34661,823.164741,16.559164,9.255721,61.778836,1.298994,20230,20230
2,16.291708,0.999988,202300002,202300015,109.34661,823.164741,16.559164,9.255721,61.778836,1.298994,20230,20230
3,16.291708,0.999988,202300003,202300015,109.34661,823.164741,16.559164,9.255721,61.778836,1.298994,20230,20230
4,16.291708,0.999988,202300004,202300015,109.34661,823.164741,16.559164,9.255721,61.778836,1.298994,20230,20230


In [27]:
def calcMaxFmeasure(df):
    match_weights = df.match_weight.unique().sort()
    print(df.head(4))
    f_measures = []

    for match_weight in match_weights:
        true_positives = float(len(df.loc[(df["match_weight"] >= match_weight) & (df["cluster_l"] == df["cluster_r"])]))
        false_positives = float(len(df.loc[(df["match_weight"] >= match_weight) & (df["cluster_l"] != df["cluster_r"])]))
        false_negatives = float(len(df.loc[(df["match_weight"] < match_weight) & (df["cluster_l"] == df["cluster_r"])]))

        f_measures.append(true_positives / (true_positives + 0.5 * (false_positives + false_negatives)))

    max_fmeasure_index = f_measures.index(max(f_measures))

    return match_weights[max_fmeasure_index]


In [24]:
print(calcMaxFmeasure(dedupe_records))

   match_weight  match_probability unique_id_l unique_id_r  bf_given_name  \
0     16.291708           0.999988   202300000   202300015      109.34661   
1     16.291708           0.999988   202300001   202300015      109.34661   
2     16.291708           0.999988   202300002   202300015      109.34661   
3     16.291708           0.999988   202300003   202300015      109.34661   

   bf_family_name   bf_dob_d  bf_dob_m   bf_dob_y  bf_gender cluster_l  \
0      823.164741  16.559164  9.255721  61.778836   1.298994     20230   
1      823.164741  16.559164  9.255721  61.778836   1.298994     20230   
2      823.164741  16.559164  9.255721  61.778836   1.298994     20230   
3      823.164741  16.559164  9.255721  61.778836   1.298994     20230   

  cluster_r  
0     20230  
1     20230  
2     20230  
3     20230  


TypeError: 'NoneType' object is not iterable

In [7]:
graphs_all = LinkageGraph(dedupe_records, prob_threshold = 0.9, min_order = 1)
df_all = graphs_all.get_measures()


TypeError: Source and target IDs must be 0-based integers, found types [dtype('O'), dtype('O')]

In [10]:
dedupe_records.head(5)

,match_weight,match_probability,unique_id_l,unique_id_r,bf_given_name,bf_family_name,bf_dob_d,bf_dob_m,bf_dob_y,bf_gender,cluster_l,cluster_r
0,15.839668,0.999983,Q109885559_0,Q109885559_11,100.074508,704.774239,61.145467,9.22181,15.847046,1.286043,Q109885559,Q109885559
1,15.839668,0.999983,Q109885559_1,Q109885559_11,100.074508,704.774239,61.145467,9.22181,15.847046,1.286043,Q109885559,Q109885559
2,15.839668,0.999983,Q109885559_10,Q109885559_11,100.074508,704.774239,61.145467,9.22181,15.847046,1.286043,Q109885559,Q109885559
3,15.839668,0.999983,Q12791743_0,Q12791743_40,100.074508,704.774239,61.145467,9.22181,15.847046,1.286043,Q12791743,Q12791743
4,15.839668,0.999983,Q12791743_2,Q12791743_40,100.074508,704.774239,61.145467,9.22181,15.847046,1.286043,Q12791743,Q12791743


In [19]:
import math

def reformatID(df_id):
    """Function to reformat IDs

    Args:
        df_id (Pandas DataFrame): Sinlge column data frame containing ID to be reformatted

    Returns:
        (Pandas DataFrame): Sinlge column data frame containing reformatted ID
    """
    col_name = df_id.columns.values[0]

    df_id = df_id.copy()

    df_id[["cluster_id", "record_index"]] = df_id[col_name].str.split("_", n = 2, expand = True)

    df_id["cluster_id"] = df_id["cluster_id"].str[1:]

    index_width = math.ceil(math.log10(df_id["record_index"].map(len).max()))
    df_id["record_index"] = df_id["record_index"].str.pad(width = index_width, fillchar='0')

    df_id[col_name] = df_id["cluster_id"] + df_id["record_index"]

    return df_id.loc[:, [col_name]]


dedupe_records["unique_id_r"] = reformatID(dedupe_records.loc[:, ["unique_id_r"]])
dedupe_records.head(5)

,match_weight,match_probability,unique_id_l,unique_id_r,bf_given_name,bf_family_name,bf_dob_d,bf_dob_m,bf_dob_y,bf_gender,cluster_l,cluster_r
0,15.839668,0.999983,109885559_0,10988555911,100.074508,704.774239,61.145467,9.22181,15.847046,1.286043,Q109885559,Q109885559
1,15.839668,0.999983,109885559_1,10988555911,100.074508,704.774239,61.145467,9.22181,15.847046,1.286043,Q109885559,Q109885559
2,15.839668,0.999983,109885559_10,10988555911,100.074508,704.774239,61.145467,9.22181,15.847046,1.286043,Q109885559,Q109885559
3,15.839668,0.999983,12791743_0,1279174340,100.074508,704.774239,61.145467,9.22181,15.847046,1.286043,Q12791743,Q12791743
4,15.839668,0.999983,12791743_2,1279174340,100.074508,704.774239,61.145467,9.22181,15.847046,1.286043,Q12791743,Q12791743


### Person data

In [2]:
Path(DEDUPE_OUTPUTS_FILES_BASE).mkdir(parents=True, exist_ok=True)

synthetic_data_filename = "max_corruptions-50_prob_mult-0.5_set-1.parquet"

synthetic_data_path = os.path.join(
        FINAL_CORRUPTED_OUTPUT_FILES_BASE, 
        synthetic_data_filename
    )

match = re.search("^max_corruptions-(\d+)_prob_mult-(\d+\.?\d*)_set-(\d+)\.parquet$", synthetic_data_filename)

distinct_entities = 20332
max_dupes = int(match.group(1))
corruption_probability_multiplier = float(match.group(2))
set_id = int(match.group(3))

out_path = os.path.join(
    DEDUPE_OUTPUTS_FILES_BASE, 
    f"""max_corruptions-{max_dupes}_prob_mult-{corruption_probability_multiplier}_set-{set_id}.parquet"""
)

if os.path.exists(out_path):
    exit

df_records = pd.read_parquet(synthetic_data_path)
df_clean = prepare_df(df_records)

linker = DuckDBLinker(df_clean, connection=":temporary:")

In [3]:
df_records.head(5)

,uncorrupted_record,cluster,given_name,family_name,dob,gender,id,corruptions_applied
0,True,Q109885559,paul,phipps,1880-03-03,male,Q109885559_0,[]
1,False,Q109885559,paul,phipps,1880-03-03,male,Q109885559_1,[]
2,False,Q109885559,paul,phipps,1880-03-03,male,Q109885559_2,[]
3,False,Q109885559,paul,phipps,1880-03-03,male,Q109885559_3,[]
4,False,Q109885559,paul,phipps,1880-03-03,male,Q109885559_4,[]


In [8]:
# linkage settings
settings = {
    "probability_two_random_records_match": get_prob_two_rnd_recs_are_match(distinct_entities, max_dupes),
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        "substr(l.given_name, 1, 2) = substr(r.given_name, 1, 2) and substr(l.family_name, 1, 2) = substr(r.family_name, 1, 2)",
        "l.dob_d = r.dob_d and l.dob_m = r.dob_m and l.dob_y = r.dob_y",
    ],
    "comparisons": [
        cl.jaro_winkler_at_thresholds(
            "given_name", [0.9, 0.7]
        ),
        cl.jaro_winkler_at_thresholds(
            "family_name", [0.9, 0.7]
        ),
        cl.exact_match("dob_d"),
        cl.exact_match("dob_m"),
        cl.exact_match("dob_y"),

        cl.exact_match("gender"),
    ],
    "retain_matching_columns": False,
    "retain_intermediate_calculation_columns": True,
    "additional_columns_to_retain": ["cluster"],
    "max_iterations": 10,
    "em_convergence": 0.01,
}

linker.initialise_settings(settings)

In [9]:
linker.count_num_comparisons_from_blocking_rule("substr(l.given_name, 1, 2) = substr(r.given_name, 1, 2) and substr(l.family_name, 1, 2) = substr(r.family_name, 1, 2)")

ParserException: Parser Error: Duplicate CTE name "__splink__df_concat"

In [ ]:
# Estimate m and u values
linker.estimate_u_using_random_sampling(target_rows=5e6)
blocking_rule = "substring(l.given_name, 1, 1)  = substring(r.given_name, 1, 1) and substring(l.family_name, 1, 2)  = substring(r.family_name, 1, 2)"
training_session_names = linker.estimate_parameters_using_expectation_maximisation(
    blocking_rule
)
blocking_rule = "l.dob_d = r.dob_d and l.dob_m = r.dob_m and l.gender = r.gender"
training_session_dob = linker.estimate_parameters_using_expectation_maximisation(
    blocking_rule
)

df_predict = linker.predict()
df_edges = df_predict.as_pandas_dataframe()

df_edges = df_edges.drop(
    [
        "match_key",
    ],
    axis=1,
)

df_edges.to_parquet(out_path, index=False)